import pandas as pd
import numpy as np
import re
import scipy.stats
import statsmodels.stats.multitest
import matplotlib.pyplot as plt
import math
import seaborn as sns
import CPTAC

In [107]:
clinical = CPTAC.get_clinical() 
somatic = CPTAC.get_somatic()
gene = 'TP53' 
protein = 'TP53'
pd.options.display.max_columns = 4000 
pd.options.display.max_rows = 4000
clinical

,Proteomics_Participant_ID,Proteomics_TMT_batch,Proteomics_TMT_plex,Proteomics_TMT_channel,Proteomics_Parent_Sample_IDs,Proteomics_Aliquot_ID,Proteomics_Tumor_Normal,Proteomics_OCT,Country,Histologic_Grade_FIGO,Myometrial_invasion_Specify,Histologic_type,Treatment_naive,Tumor_purity,Case_quality,Path_Stage_Primary_Tumor-pT,Path_Stage_Reg_Lymph_Nodes-pN,Clin_Stage_Dist_Mets-cM,Path_Stage_Dist_Mets-pM,tumor_Stage-Pathological,FIGO_stage,LVSI,BMI,Age,Diabetes,Race,Ethnicity,Gender,Tumor_Site,Tumor_Site_Other,Tumor_Focality,Tumor_Size_cm,Estrogen_Receptor,Estrogen_Receptor_%,Progesterone_Receptor,Progesterone_Receptor_%,MLH1,MLH2,MSH6,PMS2,p53,Other_IHC_specify,MLH1_Promoter_Hypermethylation,Num_full_term_pregnancies,EPIC_Bcells,EPIC_CAFs,EPIC_CD4_Tcells,EPIC_CD8_Tcells,EPIC_Endothelial,EPIC_Macrophages,EPIC_NKcells,EPIC_otherCells,CIBERSORT_B _cells _naive,CIBERSORT_B _cells _memory,CIBERSORT_Plasma _cells,CIBERSORT_T _cells _CD8,CIBERSORT_T _cells _CD4 _naive,CIBERSORT_T _cells _CD4 _memory _resting,CIBERSORT_T _cells _CD4 _memory _activated,CIBERSORT_T _cells _follicular _helper,CIBERSORT_T _cells _regulatory _(Tregs),CIBERSORT_T _cells _gamma _delta,CIBERSORT_NK _cells _resting,CIBERSORT_NK _cells _activated,CIBERSORT_Monocytes,CIBERSORT_Macrophages _M0,CIBERSORT_Macrophages _M1,CIBERSORT_Macrophages _M2,CIBERSORT_Dendritic _cells _resting,CIBERSORT_Dendritic _cells _activated,CIBERSORT_Mast _cells _resting,CIBERSORT_Mast _cells _activated,CIBERSORT_Eosinophils,CIBERSORT_Neutrophils,CIBERSORT_Absolute _score,ESTIMATE_StromalScore,ESTIMATE_ImmuneScore,ESTIMATE_ESTIMATEScore,Stemness_score,Pathway_activity_EGFR,Pathway_activity_Hypoxia,Pathway_activity_JAK.STAT,Pathway_activity_MAPK,Pathway_activity_NFkB,Pathway_activity_PI3K,Pathway_activity_TGFb,Pathway_activity_TNFa,Pathway_activity_Trail,Pathway_activity_VEGF,Pathway_activity_p53,TP53_ATM,TP53_CHEK2,TP53_MDM4,TP53_RPS6KA3,TP53_TP53,TP53_pathway,PI3K_AKT1,PI3K_AKT2,PI3K_AKT3,PI3K_DEPDC5,PI3K_DEPTOR,PI3K_INPP4B,PI3K_MAPKAP1,PI3K_MLST8,PI3K_MTOR,PI3K_NPRL2,PI3K_NPRL3,PI3K_PDK1,PI3K_PIK3CA,PI3K_PIK3CB,PI3K_PIK3R1,PI3K_PIK3R2,PI3K_PPP2R1A,PI3K_PTEN,PI3K_RHEB,PI3K_RICTOR,PI3K_RPS6,PI3K_RPS6KB1,PI3K_RPTOR,PI3K_STK11,PI3K_TSC1,PI3K_TSC2,PI3K_pathway,HRD_BRCA1,HRD_BRCA2,HRD_BRCA1_or_BRCA2,Log2_SNP,Log2_INDEL,Mutation_signature_C>A,Mutation_signature_C>G,Mutation_signature_C>T,Mutation_signature_T>C,Mutation_signature_T>A,Mutation_signature_T>G,CNV_clustering,Purity_Immune,Purity_Cancer,Purity_Stroma,MSI_status,POLE_subtype,JAK1_MS_INDEL,JAK1_Mutation,WXS_normal_sample_type,WXS_normal_filename,WXS_normal_UUID,WXS_tumor_sample_type,WXS_tumor_filename,WXS_tumor_UUID,WGS_normal_sample_type,WGS_normal_UUID,WGS_tumor_sample_type,WGS_tumor_UUID,RNAseq_R1_sample_type,RNAseq_R1_filename,RNAseq_R1_UUID,RNAseq_R2_sample_type,RNAseq_R2_filename,RNAseq_R2_UUID,miRNAseq_sample_type,miRNAseq_UUID,Methylation_available,Methylation_quality,Within_Hotspot
idx,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
S001,C3L-00006,2,5,128N,C3L-00006-01,CPT0001460012,Tumor,No,United States,FIGO grade 1,under 50 %,Endometrioid,YES,Normal,Pass,pT1a (FIGO IA),pN0,cM0,Staging Incomplete,Stage I,IA,1.0,38.88,64.0,No,White,Not-Hispanic or Latino,Female,Anterior endometrium,NaN,Unifocal,2.9,Cannot be determined,NaN,Cannot be determined,NaN,Intact nuclear expression,Intact nuclear expression,Loss of nuclear expression,Intact nuclear expression,Cannot be determined,NaN,Cannot be determined,1,0.004596,0.045126,0.075643,3.438147e-02,0.047248,0.002879,6.390000e-10,0.790127,0.010444,0.000000,0.017868,0.036816,0.0,0.088248,0.000000,0.035409,0.000000,0.000000,0.000000,0.020942,0.000000,0.024896,0.047058,0.072524,0.000000,0.006121,0.000000,0.000000,0.000000,0.009675,0.37,2821.088429,4885.608881,7706.697310,0.858619,-1.17,0.09,0.93,-0.70,0.04,1.67,-0.75,-0.01,-1.09,1.14,-0.67,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,

In [3]:
hotspot_mutations = ['p.P250L', 'p.R248W', 'p.S241C', 'p.S241del', 'p.K164E', 'p.R249S', 'p.R273H', 'p.R282W',
                    'p.S127Y', 'p.G244D', 'p.A159P']
mutated_hotspot = somatic.loc[(somatic['Location'].isin(hotspot_mutations)) & (somatic['Gene'] == gene)]
hotspot_patients = mutated_hotspot['Clinical_Patient_Key']


In [67]:
clinical['Within_Hotspot'] = False
clinical.loc[hotspot_patients, 'Within_Hotspot'] = True

hotspot = clinical.loc[clinical['Within_Hotspot'] == True]
non_hotspot = clinical.loc[clinical['Within_Hotspot'] == False]
TTest = 0
ChiSquared = 0
for feature in clinical:
    if str(clinical[feature].dtypes) == 'float64' or str(clinical[feature].dtypes) == 'int64':
        ttest = scipy.stats.ttest_ind(hotspot[feature].dropna(axis=0), non_hotspot[feature].dropna(axis = 0))
        print(feature)
        print(ttest)
        TTest += 1
    elif str(clinical[feature].dtype) == 'object':
        cat_num = 0
        cat_dict = {}
        index = 1
        chi_obs = list()
        test = 0
        
        for value in clinical[feature].dropna(axis = 0):
            if value in cat_dict:
                test += 1
            else:
                cat_num += 1
                cat_dict[value] = cat_num
                
        while index <= cat_num:
            cat_obs = 0
            for value in hotspot[feature].dropna(axis = 0):
                if cat_dict[value] == index:
                    cat_obs +=1
            index += 1
            chi_obs.append(cat_obs)
        dof = cat_num - 1
        
        
        Chi_Test = scipy.stats.chisquare(chi_obs, f_exp=None, ddof=dof, axis=0)
        print(feature)
        print(Chi_Test)
    

Proteomics_Participant_ID
Power_divergenceResult(statistic=110.99999999999999, pvalue=nan)
Proteomics_TMT_batch
Ttest_indResult(statistic=0.5417103612376254, pvalue=0.5888164733972103)
Proteomics_TMT_plex
Ttest_indResult(statistic=0.24396540137364253, pvalue=0.8075887754475264)
Proteomics_TMT_channel
Power_divergenceResult(statistic=3.0000000000000004, pvalue=nan)
Proteomics_Parent_Sample_IDs
Power_divergenceResult(statistic=141.0, pvalue=nan)
Proteomics_Aliquot_ID
Power_divergenceResult(statistic=141.0, pvalue=nan)
Proteomics_Tumor_Normal
Power_divergenceResult(statistic=36.0, pvalue=nan)
Proteomics_OCT
Power_divergenceResult(statistic=3.0, pvalue=nan)
Country
Power_divergenceResult(statistic=4.666666666666666, pvalue=nan)
Histologic_Grade_FIGO
Power_divergenceResult(statistic=0.4, pvalue=nan)
Myometrial_invasion_Specify
Power_divergenceResult(statistic=8.0, pvalue=nan)
Histologic_type
Power_divergenceResult(statistic=8.666666666666666, pvalue=nan)
Treatment_naive
Power_divergenceResu

WXS_normal_UUID
Power_divergenceResult(statistic=90.99999999999999, pvalue=nan)
WXS_tumor_sample_type
Power_divergenceResult(statistic=0.0, pvalue=nan)
WXS_tumor_filename
Power_divergenceResult(statistic=90.99999999999999, pvalue=nan)
WXS_tumor_UUID
Power_divergenceResult(statistic=90.99999999999999, pvalue=nan)
WGS_normal_sample_type
Power_divergenceResult(statistic=0.0, pvalue=nan)
WGS_normal_UUID
Power_divergenceResult(statistic=90.99999999999999, pvalue=nan)
WGS_tumor_sample_type
Power_divergenceResult(statistic=0.0, pvalue=nan)
WGS_tumor_UUID
Power_divergenceResult(statistic=90.99999999999999, pvalue=nan)
RNAseq_R1_sample_type
Power_divergenceResult(statistic=12.0, pvalue=nan)
RNAseq_R1_filename
Power_divergenceResult(statistic=105.0, pvalue=nan)
RNAseq_R1_UUID
Power_divergenceResult(statistic=105.0, pvalue=nan)
RNAseq_R2_sample_type
Power_divergenceResult(statistic=12.0, pvalue=nan)
RNAseq_R2_filename
Power_divergenceResult(statistic=105.0, pvalue=nan)
RNAseq_R2_UUID
Power_diverg

In [ ]:
p-value < 0.01
LVSI
Stemness_Score
Pathway_activity_p53
PI3K_PTEN
Mutation_signature_C>G
Mutation_signature_T>A

p-value < 0.05
BMI
CIBERSORT_T _cells _CD4 _memory _resting
CIBERSORT_NK _cells _activated
CIBERSORT_Monocytes
CIBERSORT_Macrophages _M1
CIBERSORT_Mast _cells _resting
Pathway_activity_NFkB

In [60]:
CPTAC.define("LVSI")

Binary specification of whether the specified patient has lymphovascular space invasion.


In [78]:
###Tumor Attributes

In [66]:
interest = list()
for header in columns:
    if header[0:5] == 'Tumor':
        interest.append(header)
        
        
p53_mutation = CPTAC.get_cohort_clinical(interest)
p53_mutation['Within_Hotspot'] = False
p53_mutation.loc[hotspot_patients, 'Within_Hotspot'] = True



/Users/jonjarman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/jonjarman/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [113]:
hotspot = p53_mutation.loc[p53_mutation['Within_Hotspot'] == True]
non_hotspot = p53_mutation.loc[p53_mutation['Within_Hotspot'] == False]
#hotspot = hotspot.dropna(axis=0,how = 'any')
#non_hotspsot = non_hotspot.dropna(axis=0, how = 'any')
ttest = scipy.stats.ttest_ind(hotspot['Tumor_Size_cm'], non_hotspot['Tumor_Size_cm'])
print(ttest)


Ttest_indResult(statistic=0.38204990429970553, pvalue=0.7035655006095476)


In [ ]:
###Building a Chi Table

In [103]:

chi_cat_row = np.array(clinical.index)    
chi_cat_column = np.array(clinical['Histologic_Grade_FIGO']) 
chi_cat = pd.Categorical(clinical['Histologic_Grade_FIGO'])
chi_cat

chi_cat_table = pd.crosstab(chi_cat_row, chi_cat, rownames=['id'], colnames=['FIGO'])
chi_cat_table


FIGO,S001,S002,S003,S005,S007,S008,S010,S011,S012,S014,S017,S018,S019,S020,S021,S022,S023,S024,S025,S026,S027,S028,S029,S030,S031,S032,S033,S034,S035,S036,S037,S038,S039,S040,S044,S045,S046,S047,S048,S049,S050,S051,S053,S054,S055,S056,S057,S059,S060,S061,S062,S063,S064,S065,S066,S067,S068,S069,S070,S073,S074,S075,S076,S077,S078,S079,S080,S081,S082,S083,S084,S085,S086,S087,S088,S089,S090,S091,S092,S093,S094,S095,S098,S099,S100,S101
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
FIGO grade 1,1,1,0,0,1,0,1,1,1,0,1,1,1,0,1,1,1,0,1,0,1,0,0,0,0,1,0,0,0,1,1,0,1,0,1,0,1,0,1,1,0,0,1,0,1,1,0,0,0,1,1,0,0,1,0,1,0,0,0,0,0,1,1,1,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0
FIGO grade 2,0,0,1,1,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,1,1,0,0,0,1,1,1,0,0,1,0,1,0,1,0,1,0,0,1,0,0,0,0,0,1,1,1,0,0,1,1,0,0,0,1,1,0,1,1,0,0,0,1,1,0,1,0,1,1,1,1,0,1,1,0,1,1,0,0,1,1,1,0,1
FIGO grade 3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
